#Exploratory Data Analysis on Antibiotic Resistance of *Klebsiella pneumoniae* in Europe between 2010-2020

Antibiotics are drugs that are used to treat and prevent bacterial infections.  

Bacteria strains can develop mechanisms in order to eliminate the effects of antibiotics.  This is called **antibiotic resistance** and is deemed one of the biggest threats to public health.

*Klebsiella pneumoniae* belongs to ESKAPE group pathogens, which are bacteria strains that are resistant to a number of established treatment protocols and antibiotics.  These multi-resistant strains rank among the greatest clinical challenges. 





In this notebook, exploratory data analysis will be performed on antibiotic resistance data of *Klebsiella pneumoniae* towards 5 different treatment options collected from 28 EU member states and 2 EEA countries between 2010 and 2020 

Source of this data is European Centre for Disease Prevention and Control Surveillance Atlas (https://www.ecdc.europa.eu/en/antimicrobial-resistance/surveillance-and-disease-data/data-ecdc)

Data used during this analysis can be found via the GitHub link here. 


##Data loading, cleaning and pre-processing

###Package Loading

Let's start off with loading necessary packages.  We'll use numpy and pandas in data processing and analysis stages, matplotlib and seaborn in the visualization steps and os package for accessing data.   

In [ ]:
#Loading necessary packages  
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np 
import pandas as pd
import os 
%matplotlib inline

I used Google Colab and Drive for this analysis, so I'll mount my Drive to this notebook and set working directory.  If you're running this notebook locally, you can skip Drive mounting step. 

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
os.chdir("/content/gdrive/MyDrive/Online_Lecture_Notes/Data_Analysis_w_Python_Jovian/Course_Project/data")

Mounted at /content/gdrive


###Data Loading 

Our raw data consists of antimicrobial resistance data of *Klebsiella pneumoniae* for 4 different antibiotics and for a combination of these treatments: 

1.   Carbapenems 
2.   3rd Generation Cephalosporins
3. Fluoroquinolones 
4. Aminoglycosides
5. Combined resistance (3rd gen cephalosporins + fluoroquinolones + aminoglycosides) 

We have a .csv file for each of the treatment options. We'll load each of them to individual data frames for cleaning and pre-processing. 



In [ ]:
carbapenems_df = pd.read_csv("carbapenems_ECDC_surveillance_data_Antimicrobial_resistance.csv")
thirdgen_cephalosporins_df = pd.read_csv("thirdgen_cep_ECDC_surveillance_data_Antimicrobial_resistance.csv")
fluoroquinolones_df = pd.read_csv("fluoroquinolones_ECDC_surveillance_data_Antimicrobial_resistance.csv")
aminoglycosides_df = pd.read_csv("aminoglycosides_ECDC_surveillance_data_Antimicrobial_resistance.csv")
combined_resistance_df = pd.read_csv("combined_resistance_ECDC_surveillance_data_Antimicrobial_resistance.csv")

In [ ]:
#Let's have a look at one of the dataframes
carbapenems_df.head()

,HealthTopic,Population,Indicator,Unit,Time,RegionCode,RegionName,NumValue,TxtValue
0,Antimicrobial resistance,Klebsiella pneumoniae|Carbapenems,"I - 'susceptible, increased exposure' isolates",N,2005,AT,Austria,0.000000000,NaN
1,Antimicrobial resistance,Klebsiella pneumoniae|Carbapenems,"I - 'susceptible, increased exposure' isolates",N,2005,BE,Belgium,0.000000000,NaN
2,Antimicrobial resistance,Klebsiella pneumoniae|Carbapenems,"I - 'susceptible, increased exposure' isolates",N,2005,BG,Bulgaria,0.000000000,NaN
3,Antimicrobial resistance,Klebsiella pneumoniae|Carbapenems,"I - 'susceptible, increased exposure' isolates",N,2005,CY,Cyprus,0.000000000,NaN
4,Antimicrobial resistance,Klebsiella pneumoniae|Carbapenems,"I - 'susceptible, increased exposure' isolates",N,2005,CZ,Czechia,0.000000000,NaN



To understand the data better let's check what type of data is presented in Indicator column.



In [ ]:
carbapenems_df.Indicator.unique()
#.unique() will return all the unique values from a column

array(["I - 'susceptible, increased exposure' isolates ",
       'R - resistant isolates', 'R - resistant isolates, percentage  ',
       'S - susceptible isolates', 'Total tested isolates'], dtype=object)

According to European Committee on Antimicrobial Susceptibility Testing 
[(EUCAST)](https://www.eucast.org/newsiandr/)

- **S - Susceptible, standard dosing regimen:**  when there is a high likelihood of therapeutic success using a standard dosing regimen of the agent.

- **I - Susceptible, increased exposure:** when there is a high likelihood of therapeutic success because exposure to the agent is increased by adjusting the dosing regimen or by its concentration at the site of infection.

- **R - Resistant:** when there is a high likelihood of therapeutic failure even when there is increased exposure. 

Also it seems that HealthTopic and TxtValue columns don't contain any differentiating data.  If so, these columns can be removed in the data cleaning step

In [ ]:
carbapenems_df.HealthTopic.unique() , carbapenems_df.TxtValue.unique()

(array(['Antimicrobial resistance'], dtype=object), array([nan]))

###Data Cleaning & Filtering

For our purposes, we'll only use the data in *Indicator*,*Population*, *Time*, *RegionName* and *NumValue* columns. Let's check if all datasets have those columns and remove other columns

In [ ]:
thirdgen_cephalosporins_df.columns,carbapenems_df.columns,fluoroquinolones_df.columns,aminoglycosides_df.columns,combined_resistance_df.columns

(Index(['HealthTopic', 'Population', 'Indicator', 'Unit', 'Time', 'RegionCode',
        'RegionName', 'NumValue', 'TxtValue'],
       dtype='object'),
 Index(['HealthTopic', 'Population', 'Indicator', 'Unit', 'Time', 'RegionCode',
        'RegionName', 'NumValue', 'TxtValue'],
       dtype='object'),
 Index(['HealthTopic', 'Population', 'Indicator', 'Unit', 'Time', 'RegionCode',
        'RegionName', 'NumValue', 'TxtValue'],
       dtype='object'),
 Index(['HealthTopic', 'Population', 'Indicator', 'Unit', 'Time', 'RegionCode',
        'RegionName', 'NumValue', 'TxtValue'],
       dtype='object'),
 Index(['HealthTopic', 'Population', 'Indicator', 'Unit', 'Time', 'RegionCode',
        'RegionName', 'NumValue', 'TxtValue'],
       dtype='object'))

In [ ]:
carbapenems_df = carbapenems_df[["Population","RegionName","Time","Indicator","NumValue"]]
thirdgen_cephalosporins_df = thirdgen_cephalosporins_df[["Population","RegionName","Time","Indicator","NumValue"]]
fluoroquinolones_df= fluoroquinolones_df[["Population","RegionName","Time","Indicator","NumValue"]]
aminoglycosides_df = aminoglycosides_df[["Population","RegionName","Time","Indicator","NumValue"]]
combined_resistance_df = combined_resistance_df[["Population","RegionName","Time","Indicator","NumValue"]]

In [ ]:
carbapenems_df.head()

,Population,RegionName,Time,Indicator,NumValue
0,Klebsiella pneumoniae|Carbapenems,Austria,2005,"I - 'susceptible, increased exposure' isolates",0.000000000
1,Klebsiella pneumoniae|Carbapenems,Belgium,2005,"I - 'susceptible, increased exposure' isolates",0.000000000
2,Klebsiella pneumoniae|Carbapenems,Bulgaria,2005,"I - 'susceptible, increased exposure' isolates",0.000000000
3,Klebsiella pneumoniae|Carbapenems,Cyprus,2005,"I - 'susceptible, increased exposure' isolates",0.000000000
4,Klebsiella pneumoniae|Carbapenems,Czechia,2005,"I - 'susceptible, increased exposure' isolates",0.000000000


Now we'll combine all datasets into one main set for further analysis. 

In [ ]:
combined_df = pd.concat([thirdgen_cephalosporins_df,carbapenems_df,fluoroquinolones_df,aminoglycosides_df,combined_resistance_df],axis=0)

combined_df.sample(10)

,Population,RegionName,Time,Indicator,NumValue
753,Klebsiella pneumoniae|Fluoroquinolones,Croatia,2014,R - resistant isolates,148.000000000
655,Klebsiella pneumoniae|Third-generation cephalo...,Czechia,2011,R - resistant isolates,621.000000000
1284,Klebsiella pneumoniae|Third-generation cephalo...,Greece,2016,"R - resistant isolates, percentage",72.480948340
499,Klebsiella pneumoniae|Aminoglycosides,Romania,2005,R - resistant isolates,2.000000000
397,Klebsiella pneumoniae|Third-generation cephalo...,France,2018,"I - 'susceptible, increased exposure' isolates",41.000000000
29,Klebsiella pneumoniae|Third-generation cephalo...,Austria,2006,"I - 'susceptible, increased exposure' isolates",1.000000000
149,Klebsiella pneumoniae|Third-generation cephalo...,Bulgaria,2010,"I - 'susceptible, increased exposure' isolates",4.000000000
853,Klebsiella pneumoniae|Combined resistance (thi...,Norway,2017,"R - resistant isolates, percentage",3.201024320
299,Klebsiella pneumoniae|Combined resistance (thi...,Cyprus,2015,R - resistant isolates,11.000000000
1785,Klebsiella pneumoniae|Third-generation cephalo...,Czechia,2017,S - susceptible isolates,617.000000000


We can see from the table called above, our datasets includes years other than 2010-2020.  Let's check what years are in the dataset and remove all years before 2010. 

In [ ]:
combined_df.Time.unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020])

In [ ]:
timefiltered_df = combined_df[combined_df["Time"]>=2010]
timefiltered_df.Time.unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

Let's see if there's any missing data

In [ ]:
#check if any missing countries 

countries = []
for i in timefiltered_df.RegionName.unique():
  countries.append(str(i))
print( "number of countries" ,len(countries))

for i in countries: 
  print("number of rows containing data on",i, ":", len(timefiltered_df.NumValue[timefiltered_df.RegionName==i].values))



number of countries 30
number of rows containing data on Austria : 253
number of rows containing data on Belgium : 253
number of rows containing data on Bulgaria : 253
number of rows containing data on Cyprus : 253
number of rows containing data on Czechia : 253
number of rows containing data on Germany : 253
number of rows containing data on Denmark : 253
number of rows containing data on Estonia : 253
number of rows containing data on Greece : 253
number of rows containing data on Spain : 253
number of rows containing data on Finland : 253
number of rows containing data on France : 253
number of rows containing data on Croatia : 253
number of rows containing data on Hungary : 253
number of rows containing data on Ireland : 253
number of rows containing data on Iceland : 253
number of rows containing data on Italy : 253
number of rows containing data on Lithuania : 253
number of rows containing data on Luxembourg : 253
number of rows containing data on Latvia : 253
number of rows cont

In [ ]:
#UK and Slovakia has less number of rows, indicating there's either missing years or missing indicator values 

print("years in UK data",timefiltered_df.Time.loc[(timefiltered_df.RegionName == "United Kingdom")].unique(),"\n"
      "years in Slovakia data",timefiltered_df.Time.loc[(timefiltered_df.RegionName == "Slovakia")].unique())

years in UK data [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019] 
years in Slovakia data [2011 2012 2013 2014 2015 2016 2017 2018 2019 2020]


####Handling missing data

2020 data for UK and 2010 data for Slovakia are missing.  Different approaches can be taken for filling in these data. For UK we'll replace the missing data with the data from the previous year and for Slovakia, we'll replace the missing data with 2011 data. 

In [ ]:
#while trying to replace missing 2010 data with 2009 for Slovakia, I realized data collection from Slovakia started after 2010, so I used 2011 data

#we first pull 2019 UK and 2011 Slovakia data into separate dataframes
uk2019_data = combined_df.loc[(combined_df["RegionName"]=="United Kingdom" )& (combined_df["Time"]==2019)]
slovakia2010_data=combined_df.loc[(combined_df["RegionName"]=="Slovakia")&(combined_df["Time"]==2011)]

#then change years
uk2019_data.replace({2019:2020},inplace=True)
slovakia2010_data.replace({2011:2010},inplace=True)

#and combine them with the main dataframe
timefiltered_df2=pd.concat([timefiltered_df,uk2019_data,slovakia2010_data],axis=0,ignore_index=True)

#lets have a look if we managed to successfully add all
timefiltered_df2.loc[timefiltered_df2["RegionName"]=="United Kingdom"]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,Population,RegionName,Time,Indicator,NumValue
175,Klebsiella pneumoniae|Third-generation cephalo...,United Kingdom,2010,"I - 'susceptible, increased exposure' isolates",10.000000000
205,Klebsiella pneumoniae|Third-generation cephalo...,United Kingdom,2011,"I - 'susceptible, increased exposure' isolates",10.000000000
235,Klebsiella pneumoniae|Third-generation cephalo...,United Kingdom,2012,"I - 'susceptible, increased exposure' isolates",8.000000000
265,Klebsiella pneumoniae|Third-generation cephalo...,United Kingdom,2013,"I - 'susceptible, increased exposure' isolates",6.000000000
295,Klebsiella pneumoniae|Third-generation cephalo...,United Kingdom,2014,"I - 'susceptible, increased exposure' isolates",9.000000000
...,...,...,...,...,...
1870,Klebsiella pneumoniae|Aminoglycosides,United Kingdom,2020,S - susceptible isolates,4949.000000000
2345,Klebsiella pneumoniae|Aminoglycosides,United Kingdom,2020,Total tested isolates,5406.000000000
445,Klebsiella pneumoniae|Combined resistance (thi...,United Kingdom,2020,R - resistant isolates,249.000000000
920,Klebsiella pneumoniae|Combined resistance (thi...,United Kingdom,2020,"R - resistant isolates, percentage",5.357142850


We can also remove Klebsiella pneumonia from population column to have cleaner looking charts in further steps

In [ ]:
#as the replacement is character-sensitive exact wording of the column is required

timefiltered_df2.Population.unique()

array(['Klebsiella pneumoniae|Third-generation cephalosporins',
       'Klebsiella pneumoniae|Carbapenems',
       'Klebsiella pneumoniae|Fluoroquinolones',
       'Klebsiella pneumoniae|Aminoglycosides',
       'Klebsiella pneumoniae|Combined resistance (third-generation cephalosporin, fluoroquinolones and aminoglycoside)'],
      dtype=object)

In [ ]:
timefiltered_df2.replace({"Klebsiella pneumoniae|Third-generation cephalosporins": "Third-gen cephalosporins","Klebsiella pneumoniae|Carbapenems":"Carbapenems",
                          "Klebsiella pneumoniae|Fluoroquinolones":"Fluoroquinolones","Klebsiella pneumoniae|Aminoglycosides":"Aminoglycosides",
                          "Klebsiella pneumoniae|Combined resistance (third-generation cephalosporin, fluoroquinolones and aminoglycoside)":"Combined Resistance (third-gen + fluoro + amino"},
                         inplace=True)

In [ ]:
timefiltered_df2.Population.unique()

array(['Third-gen cephalosporins', 'Carbapenems', 'Fluoroquinolones',
       'Aminoglycosides',
       'Combined Resistance (third-gen + fluoro + amino'], dtype=object)

#Exploratory Data Analysis and Visualization 

In [ ]:
timefiltered_df2.Indicator.unique()

array(["I - 'susceptible, increased exposure' isolates ",
       'R - resistant isolates', 'R - resistant isolates, percentage  ',
       'S - susceptible isolates', 'Total tested isolates'], dtype=object)